In [3]:
import gspread as gs
import streamlit as st
import pandas as pd


gc = gs.service_account_from_dict(st.secrets['gcp_service_account'])
ss = gc.open_by_key(st.secrets['portfolio_spreadsheet'].spreadsheet_key)
.worksheet(st.secrets['portfolio_spreadsheet'].worksheet_name)

In [10]:

df = pd.DataFrame(ss.get_all_records())
df.query('Portfolio == "ZEN REMIX"', inplace=True)
df

,Date,Portfolio,Account,Asset,Price,Amount,Fees,Total,Total€,Currency,Out,Profit
0,30 Apr 2021,ZEN REMIX,eToro,$,,,,598.00,,$,,
1,30 Apr 2021,ZEN REMIX,eToro,GREENBULL,598,1,,598.00,,$,,
2,30 Jul 2021,ZEN REMIX,eToro,$,,,,585.00,,$,,
3,30 Jul 2021,ZEN REMIX,eToro,GREENBULL,585,1,,585.00,,$,,
5,10 Aug 2021,ZEN REMIX,B.Direct CTO,€,,,,300.00,300€,€,,
6,1 Oct 2021,ZEN REMIX,eToro,$,,,,575.00,,$,,
7,1 Oct 2021,ZEN REMIX,eToro,GREENBULL,575,1,,575.00,,$,,
8,1 Nov 2021,ZEN REMIX,eToro,$,,,,575.00,,$,,
9,1 Nov 2021,ZEN REMIX,eToro,GREENBULL,575,1,,575.00,,$,,
10,1 Dec 2021,ZEN REMIX,eToro,$,,,,1125.00,,$,,
